# Praktikum 1

**Langkah 1**

Import Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import chi2, SelectKBest, RFE
from sklearn.metrics import accuracy_score, classification_report

**Langkah 2**

Load data dan pengelompokan data

In [3]:
df = pd.read_csv("Titanic-Dataset.csv")

# pisahkan survived
y = df["Survived"].astype(int)
X = df.drop(columns=["Survived"])

# Buat list variabel numerik dan kategorikal untuk proses seleksi fitur.
# Name tidak digunakan karena tidak relevan
num_cols = ["Age", "SibSp", "Parch", "Fare"]
cat_cols = ["Pclass", "Sex", "Embarked"]

**Langkah 3**

Ekstraksi Fitur

In [6]:
# Data Numerik
num_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

# Data kategorikal
cat_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

**Langkah 3.1**

Feature Construction

In [8]:
# Buat fitur FamilySize
X["FamilySize"] = X["SibSp"].fillna(0) + X["Parch"].fillna(0) +1

# Tambahkan FamilySize pada kelompok numerikal
preprocess = ColumnTransformer([
    ("num", num_tf, num_cols + ["FamilySize"]),
    ("cat", cat_tf, cat_cols),
])

**Langkah 4**

Seleksi Fitur

In [9]:
# Analisis ANOVA
from sklearn.feature_selection import f_classif
selector_filter = SelectKBest(score_func=f_classif, k=5)

# Buat pipeline final
pipe_filter = Pipeline([
    ("prep", preprocess),
    ("sel", selector_filter),
    ("clf", LogisticRegression(max_iter=1000)),
])

**Langkah 5**

Uji dengan model

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

pipe_filter.fit(X_train, y_train)
pred = pipe_filter.predict(X_test)
print("===== FILTER (ANOVA) + LR ====")
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

===== FILTER (ANOVA) + LR ====
Accuracy: 0.8100558659217877
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       110
           1       0.79      0.70      0.74        69

    accuracy                           0.81       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.81      0.81      0.81       179

